In [369]:
# Imorting libraries 

import requests
import pandas as pd 
import json
import calendar
import hvplot.pandas
from pathlib import Path
import seaborn as sns
%matplotlib inline
from bokeh.models.formatters import DatetimeTickFormatter
pd.set_option("display.max_rows", 10, "display.max_columns", None)



series_list1 = {
'LAUST010000000000003':'Alabama', 'LAUST020000000000003':'Alaska', 'LAUST040000000000003':'Arizona', 'LAUST050000000000003':'Arkansas', 'LAUST060000000000003':'California', 
'LAUST080000000000003':'Colorado', 'LAUST090000000000003':'Connecticut', 'LAUST100000000000003':'Delaware', 'LAUST120000000000003':'Florida', 'LAUST130000000000003':'Georgia', 
'LAUST150000000000003':'Hawaii', 'LAUST160000000000003':'Idaho', 'LAUST170000000000003':'Illinois', 'LAUST180000000000003':'Indiana', 
'LAUST190000000000003':'Iowa', 'LAUST200000000000003':'Kansas', 'LAUST210000000000003':'Kentucky', 'LAUST220000000000003':'Louisiana', 'LAUST230000000000003':'Maine', 
'LAUST240000000000003':'Maryland', 'LAUST250000000000003':'Massachusetts', 'LAUST260000000000003':'Michigan', 'LAUST270000000000003':'Minnesota', 'LAUST280000000000003':'Mississippi',
'LAUST290000000000003':'Missouri'} 
series_list2 = {
'LAUST300000000000003':'Montana', 'LAUST310000000000003':'Nebraska', 'LAUST320000000000003':'Nevada', 'LAUST330000000000003':'New Hampshire', 'LAUST340000000000003':'New Jersey', 
'LAUST350000000000003':'New Mexico', 'LAUST360000000000003':'New York', 'LAUST370000000000003':'North Carolina', 'LAUST380000000000003':'North Dakota', 'LAUST390000000000003':'Ohio', 'LAUST400000000000003':'Oklahoma', 
'LAUST410000000000003':'Oregon', 'LAUST420000000000003':'Pennsylvania', 'LAUST440000000000003':'Rhode Island', 'LAUST450000000000003':'South Carolina', 'LAUST460000000000003':'South Dakota', 'LAUST470000000000003':'Tennessee', 
'LAUST480000000000003':'Texas', 'LAUST490000000000003':'Utah', 'LAUST500000000000003':'Vermont', 'LAUST510000000000003':'Virginia', 
'LAUST530000000000003':'Washington', 'LAUST540000000000003':'West Virginia', 'LAUST550000000000003':'Wisconsin', 'LAUST560000000000003':'Wyoming',
}
df = pd.DataFrame(columns= ["State","Year","Month","Unemployment Rate"])


In [370]:
headers = {'Content-type': 'application/json'}
data1 = json.dumps({"seriesid": list(series_list1.keys()),"startyear":"2019", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data1, headers=headers)
json_data1 = json.loads(p.text)
data2 = json.dumps({"seriesid": list(series_list2.keys()),"startyear":"2019", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data2, headers=headers)
json_data2 = json.loads(p.text)

for series in json_data1['Results']['series']:
    state = series_list1[series['seriesID']]
    for item in series['data']:
        year = item['year']
        month = calendar.month_name[int(item['period'][1:3])]
        value = item['value']
        df = df.append({'State':state,'Year':year,'Month':month,'Unemployment Rate':value},ignore_index=True)

for series in json_data2['Results']['series']:
    state = series_list2[series['seriesID']]
    for item in series['data']:
        year = item['year']
        month = calendar.month_name[int(item['period'][1:3])]
        value = item['value']
        df = df.append({'State':state,'Year':year,'Month':month,'Unemployment Rate':value},ignore_index=True)
df

,State,Year,Month,Unemployment Rate
0,Alabama,2021,December,2.6
1,Alabama,2021,November,2.6
2,Alabama,2021,October,2.9
3,Alabama,2021,September,3.0
4,Alabama,2021,August,3.7
...,...,...,...,...
1795,Wyoming,2019,May,3.3
1796,Wyoming,2019,April,3.4
1797,Wyoming,2019,March,4.0
1798,Wyoming,2019,February,3.9


In [371]:
df_2021 = df.loc[df['Year'] == '2021'].copy()
df_2021['Year Month'] = df_2021['Year'] +' '+ df_2021['Month']
df_2021 = df_2021[['Year Month', 'State','Unemployment Rate']]
df_2021.set_index('Year Month')
df_2021


,Year Month,State,Unemployment Rate
0,2021 December,Alabama,2.6
1,2021 November,Alabama,2.6
2,2021 October,Alabama,2.9
3,2021 September,Alabama,3.0
4,2021 August,Alabama,3.7
...,...,...,...
1771,2021 May,Wyoming,4.7
1772,2021 April,Wyoming,5.0
1773,2021 March,Wyoming,5.4
1774,2021 February,Wyoming,6.0


In [372]:
df_2021.sort_index(inplace=True,ascending=False)
df_2021 = df_2021.astype({'Unemployment Rate':'float'})
df_2021.hvplot(y='Unemployment Rate', x = 'Year Month',groupby='State', rot = 90)

:DynamicMap   [State]
   :Curve   [Year Month]   (Unemployment Rate)

In [373]:
# Setting up the path to the CSV file
csv_path = Path("../FInCen CSV Data/FinCen SAR Stats 2021 (Months).csv")

# Reading the csv file
csv_file = pd.read_csv(csv_path, index_col = "Year Month", thousands=',')
csv_file


,State,Industry,Suspicious Activity,Count
Year Month,,,,
2021 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Credit/Debit Card,2
2021 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Other Fraud (Type),2
2021 January,Alabama,Casino/Card Club - Tribal Authorized Casino,[Total],4
2021 January,Alabama,Depository Institution,ACH,286
2021 January,Alabama,Depository Institution,Advance Fee,3
...,...,...,...,...
2021 December,Wyoming,Securities/Futures,[Total],1
2021 December,Wyoming,[Total],[Total],73
2021 December,[Total],[Total],[Total],160929


In [374]:
#Using "loc" to remove the lines that have "Totals"

csv_file = csv_file.loc[csv_file["Industry"]!="[Total]" ]
csv_file = csv_file.loc[csv_file["Suspicious Activity"]!="[Total]" ]
csv_file = csv_file.loc[csv_file["State"]!="[Total]" ]
csv_file

,State,Industry,Suspicious Activity,Count
Year Month,,,,
2021 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Credit/Debit Card,2
2021 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Other Fraud (Type),2
2021 January,Alabama,Depository Institution,ACH,286
2021 January,Alabama,Depository Institution,Advance Fee,3
2021 January,Alabama,Depository Institution,Business Loan,7
...,...,...,...,...
2021 December,Wyoming,Depository Institution,Wire,6
2021 December,Wyoming,Money Services Business (MSB),Credit/Debit Card,1
2021 December,Wyoming,Money Services Business (MSB),Other Fraud (Type),18


In [375]:
csv_file = csv_file.reset_index()
csv_file["Year Month"] = pd.to_datetime(csv_file["Year Month"], infer_datetime_format = True)


grouped_stats = csv_file[["State", "Count"]].copy()
grouped_stats = grouped_stats.groupby(["State"]).sum()
grouped_stats


,Count
State,
APO / DPO / FPO,78
Alabama,27125
Alaska,1531
American Samoa,13
Arizona,14840
...,...
Virginia,124754
Washington,19131
West Virginia,8078


In [376]:
mapped_data = grouped_stats.hvplot.bar(
     rot = 90,
    frame_width = 700, frame_height = 500,
    label = "FinCen Fraud SAR Reporting by State in 2021"
)
mapped_data

:Bars   [State]   (Count)

In [377]:

df_2021 = df_2021.reset_index()
df_2021 = df_2021.drop(columns ='index')
grouped_2021 = df_2021.groupby(['Year Month', 'State']).sum()
grouped_2021





Unemployment Rate
Year Month     State                           
2021 April     Alabama                      3.3
               Alaska                       7.5
               Arizona                      5.5
               Arkansas                     4.2
               California                   8.3
...                                         ...
2021 September Virginia                     3.4
               Washington                   4.6
               West Virginia                3.8
               Wisconsin                    3.0
               Wyoming                      3.4

[600 rows x 1 columns]

In [378]:
industry_by_ftype = csv_file.groupby(['Industry', 'Suspicious Activity']).sum() 
industry_by_ftype.hvplot.bar(stacked=True, frame_height=500, width=1050, rot=90, legend='top_left', yformatter='%.0f', label= 'Fraud Report by Industry Highlighted by the Type 2021')

:Bars   [Industry,Suspicious Activity]   (Count)

In [379]:
csv_file_2 = csv_file.copy()
csv_file_2['Year Month'] = csv_file_2['Year Month'].astype(str)

In [380]:
sar_grouped_stats = csv_file_2[['State', 'Count', 'Year Month']].copy()
sar_grouped_stats = sar_grouped_stats.groupby(['State','Year Month',]).sum()
sar_grouped_stats.hvplot.bar(stacked=True, frame_height=500, width=1050, rot=90, legend='left', yformatter='%.0f', label= 'Fraud Reports by State Highlighted by Month 2021')

:Bars   [State,Year Month]   (Count)

In [381]:
csv_file

,Year Month,State,Industry,Suspicious Activity,Count
0,2021-01-01,Alabama,Casino/Card Club - Tribal Authorized Casino,Credit/Debit Card,2
1,2021-01-01,Alabama,Casino/Card Club - Tribal Authorized Casino,Other Fraud (Type),2
2,2021-01-01,Alabama,Depository Institution,ACH,286
3,2021-01-01,Alabama,Depository Institution,Advance Fee,3
4,2021-01-01,Alabama,Depository Institution,Business Loan,7
...,...,...,...,...,...
16210,2021-12-01,Wyoming,Depository Institution,Wire,6
16211,2021-12-01,Wyoming,Money Services Business (MSB),Credit/Debit Card,1
16212,2021-12-01,Wyoming,Money Services Business (MSB),Other Fraud (Type),18
16213,2021-12-01,Wyoming,Money Services Business (MSB),Wire,4


In [382]:
combined_df = pd.concat([grouped_2021, grouped_stats], axis="columns", join="inner")
combined_df

,,Unemployment Rate,Count


In [383]:
csv_file = csv_file.set_index("Year Month")
csv_file = csv_file.drop(columns= 'Suspicious Activity')
csv_file

,State,Industry,Count
Year Month,,,
2021-01-01,Alabama,Casino/Card Club - Tribal Authorized Casino,2
2021-01-01,Alabama,Casino/Card Club - Tribal Authorized Casino,2
2021-01-01,Alabama,Depository Institution,286
2021-01-01,Alabama,Depository Institution,3
2021-01-01,Alabama,Depository Institution,7
...,...,...,...
2021-12-01,Wyoming,Depository Institution,6
2021-12-01,Wyoming,Money Services Business (MSB),1
2021-12-01,Wyoming,Money Services Business (MSB),18


In [384]:
csv_file = csv_file.loc[csv_file["Industry"] == 'Depository Institution']
csv_file

,State,Industry,Count
Year Month,,,
2021-01-01,Alabama,Depository Institution,286
2021-01-01,Alabama,Depository Institution,3
2021-01-01,Alabama,Depository Institution,7
2021-01-01,Alabama,Depository Institution,665
2021-01-01,Alabama,Depository Institution,23
...,...,...,...
2021-12-01,Wyoming,Depository Institution,1
2021-12-01,Wyoming,Depository Institution,6
2021-12-01,Wyoming,Depository Institution,1


In [385]:
csv_file = csv_file.reset_index()
csv_file['Year Month'] = pd.to_datetime(csv_file['Year Month'], infer_datetime_format = True)
csv_file['Count'] = csv_file['Count'].astype(float)

csv_file.dtypes

Year Month    datetime64[ns]
State                 object
Industry              object
Count                float64
dtype: object

In [386]:
grouped_stats_state = csv_file[["State", "Count", "Year Month"]]

grouped_stats_state = grouped_stats_state.groupby(['Year Month', 'State']).sum()
grouped_stats_state

Count
Year Month State                  
2021-01-01 Alabama          1453.0
           Alaska            101.0
           American Samoa      1.0
           Arizona           708.0
           Arkansas          346.0
...                            ...
2021-12-01 Virginia        11004.0
           Washington       1088.0
           West Virginia     962.0
           Wisconsin         743.0
           Wyoming            49.0

[687 rows x 1 columns]

In [387]:
grouped_stats_ym = csv_file.groupby(['Year Month']).sum()
grouped_stats_ym

,Count
Year Month,
2021-01-01,81186.0
2021-02-01,83913.0
2021-03-01,95883.0
2021-04-01,90782.0
2021-05-01,92555.0
...,...
2021-08-01,121469.0
2021-09-01,122066.0
2021-10-01,114214.0


In [388]:
grouped_stats_ym = grouped_stats_ym.rename(columns = {'Count' : 'Total'})
grouped_stats_ym

,Total
Year Month,
2021-01-01,81186.0
2021-02-01,83913.0
2021-03-01,95883.0
2021-04-01,90782.0
2021-05-01,92555.0
...,...
2021-08-01,121469.0
2021-09-01,122066.0
2021-10-01,114214.0


In [389]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)
grouped_stats_state = grouped_stats_state.rename(columns = {'Count' : 'Reports'})
grouped_stats_state

Reports
Year Month State                  
2021-01-01 Alabama          1453.0
           Alaska            101.0
           American Samoa      1.0
           Arizona           708.0
           Arkansas          346.0
...                            ...
2021-12-01 Virginia        11004.0
           Washington       1088.0
           West Virginia     962.0
           Wisconsin         743.0
           Wyoming            49.0

[687 rows x 1 columns]

In [390]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)
fraud_data = pd.merge(grouped_stats_state, grouped_stats_ym, left_on = 'Year Month',right_index=True)
fraud_data

Reports     Total
Year Month State                            
2021-01-01 Alabama          1453.0   81186.0
           Alaska            101.0   81186.0
           American Samoa      1.0   81186.0
           Arizona           708.0   81186.0
           Arkansas          346.0   81186.0
...                            ...       ...
2021-12-01 Virginia        11004.0  119089.0
           Washington       1088.0  119089.0
           West Virginia     962.0  119089.0
           Wisconsin         743.0  119089.0
           Wyoming            49.0  119089.0

[687 rows x 2 columns]

In [391]:
fraud_data['Reporting Rate'] = round((fraud_data['Reports']/fraud_data['Total'])*100,3)
fraud_data

Reports     Total  Reporting Rate
Year Month State                                            
2021-01-01 Alabama          1453.0   81186.0           1.790
           Alaska            101.0   81186.0           0.124
           American Samoa      1.0   81186.0           0.001
           Arizona           708.0   81186.0           0.872
           Arkansas          346.0   81186.0           0.426
...                            ...       ...             ...
2021-12-01 Virginia        11004.0  119089.0           9.240
           Washington       1088.0  119089.0           0.914
           West Virginia     962.0  119089.0           0.808
           Wisconsin         743.0  119089.0           0.624
           Wyoming            49.0  119089.0           0.041

[687 rows x 3 columns]

In [392]:
fraud_data_rate = fraud_data[['Reporting Rate']].copy()
formatter = DatetimeTickFormatter(months='%b %Y')
fraud_data_rate.hvplot(groupby='State' ,yformatter = '%.3f', xformatter=formatter)

:DynamicMap   [State]
   :Curve   [Year Month]   (Reporting Rate)

In [393]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)
fraud_data_rate

Reporting Rate
Year Month State                         
2021-01-01 Alabama                  1.790
           Alaska                   0.124
           American Samoa           0.001
           Arizona                  0.872
           Arkansas                 0.426
...                                   ...
2021-12-01 Virginia                 9.240
           Washington               0.914
           West Virginia            0.808
           Wisconsin                0.624
           Wyoming                  0.041

[687 rows x 1 columns]